Import das bibliotecas necessárias para realizar os procedimentos

In [ ]:
import MySQLdb
import sqlalchemy
import pandas as pd 
import os

Configurando a conexão com o banco de dados

In [ ]:
#SQL Connection 
user = 'jonathas' #usuário
password = '123456' #senha
host = 'localhost' #manter como está
database = 'projeto2IGTI' #schema
string_conexao = f'mysql://{user}:{password}@{host}/{database}' #url de conexão com o bacanco de dados, não mudar.

Realizando a conexão com o banco de dados 

In [ ]:
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()



Nesse momento vamos ler os arquivos que estão dentro da pasta chamada base.

In [ ]:
#Criar Lista de arquivos
lista_df = [] # cria uma lista
lista_path = os.listdir('./base') #mostra qual a pasta
lista_path[:5] #mostra os 5 primeiro arquivos que estão dentro da pasta base

#ler os arquivos a armazena num dataframe
for arquivo in lista_path: # Esse for vai percorrer todos os arquivos dentro da pasta base
    local_arquivo = os.path.join('./base', arquivo)
    df = pd.read_csv(local_arquivo, sep = '|', encoding='latin-1') #o conteúdo dos arquivos está separado por um  |, para ter certerza, abra um dos arquivos e observe qual o separador.
    lista_df.append(df) 
df_pesquisa = pd.concat(lista_df) # Atribui os dados coletados em um dataframe

df_pesquisa[:5] #mostra as 5 primeiras linhas do dataframe gerado




Inicialmente vamos trabalhar os dados que não possuem dependecias de outras tabelas.
Essa tal independência pode ser observada no schema do enunciado do trabalho.

In [ ]:
#Inserir dados no banco
#Para pessoas, podemos observar que a tabela de pessoas possui as seguintes características cod_pessoa, genero e data_nascimento.
#O que fazemos aqui é criar um dataframe de pessoas com essses dados, onde selecionamos as colunas correspondentes no df_pesquisa
df_pessoas = df_pesquisa[['cod_pessoa', 'genero', 'data_nascimento']].drop_duplicates() #o drop_duplicates remove qualquer conteúdo que esteja duplicado

#Podemos observar que o animal só tem a espécie dele para ser armazenada, que no caso está referida como "animal_estimacao"
#Como é apenas uma coluna que será armazenada, iremos usar uma lista, se fossem mais, utilizaríamos um dataframe.
animais = list(df_pesquisa.animal_estimacao.unique()) #Como o nome dos animais se repete ao longo das linhas da pesquisa, utilizamos a fução unique() para selecionar apenas os valores unicos, sem repetições.

#A mesma lógica utilizada com os animais pode ser utilizada com hobbies, bebidas e climas.
hobbies = list(df_pesquisa.hobbies.unique())
bebidas = list(df_pesquisa.bebida_favorita.unique())
climas = list(df_pesquisa.clima.unique())



Vamos verificar o dataframe de pessoas e as listas de animais, hobbies, bebidas e climas.
Para isso remova a tag comentário '#' do comando para visualizar cada uma das variáveis.

In [ ]:
df_pessoas[:5] #mostra as 5 primeiras linhas
#animais
#hobbies
#bebidas
#climas


A partir desse ponto, se supõe que o banco de dados está conectado perfeitamente e que as tabelas já estão criadas.
Caso não esteja, verifique o arquivo scriptPesquisa.sql para criar as tabelas e seu banco de acordo com o enunciado da questão.



Como df_pessoas é um dataframe, utilizaremos a seguinte lógica para inserir as pessoas no banco de dados:

In [ ]:
#Inserir pessoas
for pessoa in df_pessoas.itertuples(): #Um for que vai criar uma variável temporária do tipo pessoa que vai percorrer todo o dataframe df_pessoas
    #observando o dataframe df_pessoas podemos notar que, de acordo com as colunas, cada pessoa possui essas três características abaixo
    cod_pessoa = pessoa.cod_pessoa	 #pessoa.cod_pessoa vai referenciar inicialmente o cod_pessoa da pessoa[1] no dataframe de pessoas, e assim por diante.
    genero = pessoa.genero #o mesmo pensamento acima serve para as pessoa.genero e pessoa.data_nascimento
    nascimento = pessoa.data_nascimento
    #A partir daqui, guardamos três valores, cod_pessoa, genero e nascimento.
    try:
        #Nessa primeira etapa, vamos inserir na tabela "pessoa" nas colunas cod_pessoa, genero e data_nascimento as variaveis que guardamos anteriormente, cod_pessoa, genero e nascimento
        querey = f"""insert into pessoa (cod_pessoa, genero, data_nascimento )
                     values ('{cod_pessoa}',  '{genero}', '{nascimento}')"""      
        conn.execute(querey) #enviar soliciatação par inserir      
        print(f"Registro inserido com sucesso: {cod_pessoa}") #Se ocorrer tudo bem
    except Exception as e:        
        print(f"Não foi possível inserir o regristro {cod_pessoa}. O erro foi encontrado foi: {e}") #Se houver algum erro
print("Pessoas Inseridas")

Agora vamos inserir os dados que estão nas listas: animais, hobbies, climas e bebidas

In [ ]:
#Inserir animais
#A lógica é bem parecida com a de pessoas, mas como é apenas uma lista, só iremos inserir um valor
for a in animais: #Nesse caso, como existe apenas um valor, a variável temporária 'a' que está sendo criada no for irá assumir o valor contido em casa linha da lista animais
    try:
        #Nessa primeira etapa, vamos inserir na tabela "animal_estimacao" na coluna animal, o valor de a.
        querey = f"""insert into animal_estimacao (animal) 
                     values ('{a}')"""
        conn.execute(querey)#enviar soliciatação par inserir      
    except Exception as e:
        print(f"Não foi possível inserir o regristro {a}. O erro foi encontrado foi: {e}")  
print("Animais Inseridos")

A mesma lógica da lista de animais se aplica para as lista de hobbies, climas e bebidas

In [ ]:
#Inserir Hobbies
for h in hobbies:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "hobbie" na coluna hobbie, o valor de h.
        querey = f"""insert into hobbie (hobbie) 
                     values ('{h}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {h}. O erro foi encontrado foi: {e}")  
print("Hobbies inseridos")


In [ ]:
#Inserir Bebidas
for b in bebidas:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "bebida" na coluna bebida, o valor de b.
        querey = f"""insert into bebida (bebida) 
                     values ('{b}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {b}. O erro foi encontrado foi: {e}")  
print("Bebidas inseridas")

In [ ]:
#Inserir Clima
for c in climas:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "clima" na coluna clima, o valor de c.
        querey = f"""insert into clima (clima) 
                     values ('{c}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {c}. O erro foi encontrado foi: {e}")  
print("Climas inseridas")

Agora as variáveis estão todas inseridas no banco de dados, precisamos agora inserir no banco os dados relacionados da tabela de pesquisa. Para tal, precisamos ler as nossas tabelas que estão guardadas no banco.

Nesse ponto você pode se perguntar: Por qual motivo preciso fazer isso? Elas já não estavam armazenadas aqui no código?

Então, lembra que você criou colunas chamadas de cod_de_alguma coisa?

Para as tabelas de clima, bebida, hobbie e animal nós não inserimos um valor correspondente ao cod_clima, cod_bebida, cod_hobbie e cod_animal.

O que aconteceu então?

Essas colunas foram configuradas no mysql como auto increment e a medida que linhas novas são inseridas elas são incrementadas.

Já para a a tabela de pessoas, já existia um cod_pessoa na nossa tabela de pesquisas.

Leitura das tabelas do banco de dados

In [ ]:
df_animais = pd.read_sql('animal_estimacao', con=conn) #ler a tabela animal_estimacao
df_bebidas = pd.read_sql('bebida', con=conn) #ler a tabela bebida
df_climas = pd.read_sql('clima', con=conn) #ler a tabela clima
df_pessoas = pd.read_sql('pessoa', con=conn) #ler a tabela pessoa
df_hobbies = pd.read_sql('hobbie', con=conn) #ler a tabela hobbie

Observe que para os animais, bebidas, climas e hobbies nós inserimos listas mas recebemos data_frames.

Qual o motivo disso?

Lembra das colunas cod_clima, cod_bebida, cod_hobbie e cod_animal? Então, elas foram auto incrementadas dentro do mysql e agora foram lidas.

Observe cada um dos data_frames lidos do banco para fazer uma analogia ao que foi dito.

In [ ]:
df_bebidas

In [ ]:
df_animais


In [ ]:
df_climas

In [ ]:
df_pessoas[:5] #mostra as 5 primeiras

In [ ]:
df_hobbies

In [ ]:
#Para facilitar o entendimento criei uma copia do df_pesquisa
#Cópia do Banco
df_pesquisa2 = df_pesquisa.copy()

Agora que já temos as tabelas com seus respectivos códigos, precisamos substituir nas colunas do dataframe df_pesquisa os valores dos códigos das nossas tabelas.

Para isso criamos alguns dicionários para fazer essa tal substituição.

Para facilitar o entendimento, vamos observar a primeira pesquisa:


In [ ]:
df_pesquisa2[:1]

O que precisamos fazer é o seguinte substituir o valor de:

 peixe pelo seu código 1

 Praticar esporte por seu codigo 1

 frio pelo seu código 1

 cerveja pelo seu código 1

In [ ]:
#Usando Dicionários para substituir os valores pelos codigos
#Animais
#Com as funções abaixo é possível criar um dicionário de para os nossos dataframes e substitui-los no df_pesquisa

#Criando dicionário
dict_animais = df_animais.set_index(df_animais.animal)['cod_animal_estimacao'].to_dict()
#Substituindo no dataframe df_pesquisa
df_pesquisa.animal_estimacao = df_pesquisa2.animal_estimacao.map(dict_animais) 
#Observando o que fizemos agora:
df_pesquisa[:5]#mostra as 5 primeiras linhas

#De forma análoga fazemos para os demais dataframes:
#Bebidas
dict_bebidas = df_bebidas.set_index(df_bebidas.bebida)['cod_bebida'].to_dict()
df_pesquisa.bebida_favorita = df_pesquisa2.bebida_favorita.map(dict_bebidas) 

#Hobbies
dict_hobbies = df_hobbies.set_index(df_hobbies.hobbie)['cod_hobbie'].to_dict()
df_pesquisa.hobbies = df_pesquisa2.hobbies.map(dict_hobbies) 

#Clima
dict_climas = df_climas.set_index(df_climas.clima)['cod_clima'].to_dict()
df_pesquisa.clima = df_pesquisa2.clima.map(dict_climas) 

Nosso dataframe df_pesquisa ficou dessa forma agora:

In [ ]:
df_pesquisa

 E observando que os dataframe df_pesquisa possui as colunas cod_pessoa, genero e data_nascimento e que o dataframe df_pessoas possui as mesmas colunas, podemos removê-las e referenciá-las com o cod_pessoa quando fizermos pesquisas no mysql.

In [ ]:

#Criei uma variável nova apenas por questões didáticas para comparação entre elas
#Selecionando apenas as colunas de interesse do dataframe df_pesquisa
df_final = df_pesquisa[['cod_pessoa', 'animal_estimacao', 'clima', 'bebida_favorita', 'hobbies', 'data_coleta']]
df_final[:5]#5 primeiras linhas


In [ ]:
#Aqui estamos renomeando as colunas do dataframe, novamente com uma nova vairável para questões didáticas de comparação entre elas:
df_final2 = df_final.rename(columns={'animal_estimacao' : 'cod_animal_estimacao', 'clima' : 'cod_clima', 'bebida_favorita' : 'cod_bebida', 'hobbies' : 'cod_hobbie', 'data_coleta' : 'data_pesquisa'})
df_final2[:5]#5 primeiras linhas

Nas vídeo aulas, o professor usa um script mysql para inserir os dados de pesquisa, exportando o o dataframe df_final2, no entanto eu preri inserir de maneira análoga ao df_pessoas.

In [ ]:
for p in df_final2.itertuples():#Um for que vai criar uma variável temporária p do tipo df_final que vai percorrer todo o dataframe df_final
    #observando o dataframe df_final2 podemos notar que, de acordo com as colunas, cada pesquisa possui essas 6 características abaixo
    cod_pessoa = p.cod_pessoa	
    cod_clima = p.cod_clima
    data_pesquisa = p.data_pesquisa
    cod_animal_estimacao = p.cod_animal_estimacao
    cod_bebida = p.cod_bebida
    cod_hobbie = p.cod_hobbie
    #Aqui guardamos as variaveis cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie (6 variáveis para 6 colunas)
    try:
        #Nessa primeira etapa, vamos inserir na tabela "pesquisa" nas colunas cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie as variaveis que guardamos anteriormente, cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie.
        querey = f"""insert into pesquisa (cod_pessoa, cod_clima, data_pesquisa, cod_animal_estimacao, cod_bebida, cod_hobbie) 
                     values ('{cod_pessoa}',  '{cod_clima}', '{data_pesquisa}', '{cod_animal_estimacao}', '{cod_bebida}', '{cod_hobbie}')"""      
        conn.execute(querey)      
        print(f"Registro inserido com sucesso: {cod_pessoa}")
    except Exception as e:        
        print(f"Não foi possível inserir o regristro {cod_pessoa}. O erro foi encontrado foi: {e}")
print("Pesquisas Inseridas")